In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
import dice_equations as deq

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [2]:
# defines functions
def attack_damage(ph, pc, dc):
    return ph + pc*dc

def attack_damage_adv(ph, pc, dc):
    return ph*(2 - ph - 2*pc) + dc*pc*(2 - pc)

def attack_damage_dis(ph, pc, dc):
    return ph*(ph + 2*pc) + dc*pc*pc

def attack_bonus_adv(ph, pc, dc):
    d = attack_damage(ph, pc, dc)
    da = attack_damage_adv(ph, pc, dc)
    return 20*(da - d) # 20*(ph*(1 - ph - 2*pc) + dc*pc*(1 - pc))

def attack_bonus_dis(ph, pc, dc):
    d = attack_damage(ph, pc, dc)
    dd = attack_damage_dis(ph, pc, dc)
    return 20*(dd - d)  # 20*(ph*(ph + 2*pc - 1) + dc*pc*(pc - 1))

def save_damage(pf, ds):
    return (1 - ds)*pf + ds

def save_damage_dis(pf, ds):
    return (1 - ds)*pf*(2 - pf) + ds

In [3]:
# plots the effective increase in AB from having advantage as a function of your chance to hit.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='hit + crit probability',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='$\Delta \, AB$',
            range=[0, 7],
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            orientation='h',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
            itemclick=False, 
            itemdoubleclick=False,
            entrywidthmode='fraction',
            entrywidth=0.51,
        )
    )
)

# plot data
pc = 0.05
ph_norm = 0.60
ph = np.linspace(0.0, 1 - 2*pc, int(20*(1 - 2*pc)) + 1)
for dc, yp in zip([1.5, 2.0],[0.955,0.895]):
    fig.add_trace(go.Scatter(
        x=ph+0.05, 
        y=attack_bonus_adv(ph, pc, dc),
        mode='lines+markers', 
        marker=dict(size=8),
        unselected=dict(marker=dict(opacity=0.0, size=0)),
        selectedpoints=[12],
        name=f'$d_c = {dc:.1f}$',
        hovertemplate=
                'probability %{x:.2f}<br>'+
                'delta AB %{y:.1f}'+
                '<extra></extra>'
    ))


fig.add_trace(go.Scatter(
    x=2*[ph_norm + 0.05], 
    y=[0,7],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    #name='a properly long name to take up space',
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-delta-ab-advantage-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-delta-ab-advantage-small')

In [6]:
# plots the effective decrease in AB from having disadvantage as a function of your chance to hit.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='hit + crit probability',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='$\Delta \, AB$',
            range=[-6, 0],
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
            itemclick=False, itemdoubleclick=False,
            tracegroupgap=0,
        )
    )
)

# plot data
pc = 0.05
ph_norm = 0.60
ph = np.linspace(0.0, 1 - 2*pc, int(20*(1 - 2*pc)) + 1)
for dc in [1.5]:
    fig.add_trace(go.Scatter(
        x=ph+0.05, 
        y=attack_bonus_dis(ph, pc, dc),
        mode='lines+markers', 
        #line=dict(color='black', dash='dash'),
        marker=dict(size=8),
        unselected=dict(marker=dict(opacity=0.0, size=0)),
        selectedpoints=[12],
        name=f'$d_c = {dc:.1f}$',
        showlegend=False,
        hovertemplate=
                'probability %{x:.2f}<br>'+
                'delta AB %{y:.1f}'+
                '<extra></extra>'
    ))


fig.add_trace(go.Scatter(
    x=2*[ph_norm + 0.05], 
    y=[-6,0],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-delta-ab-disadvantage-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-delta-ab-disadvantage-small')

In [4]:
# plots the effective increase in DC from having disadvantage as a function of your chance to fail the save.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='fail save probability',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='$\Delta \, DC$',
            range=[0, 6],
            tick0=0, dtick=1.0,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            orientation='h',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
            itemclick=False, 
            itemdoubleclick=False,
            entrywidthmode='fraction',
            entrywidth=0.51,
        )
    )
)

# plot data
ds = 0.5
pf_norm = 0.65
pf = np.linspace(0.0, 1.00, 21)
for ds, yp in zip([0.5, 0.0], [0.955,0.895]):
    fig.add_trace(go.Scatter(
        x=pf, 
        y=20*(save_damage_dis(pf, ds) - save_damage(pf, ds)),
        mode='lines+markers', 
        marker=dict(size=8),
        unselected=dict(marker=dict(opacity=0.0, size=0)),
        selectedpoints=[13],
        name=f'$d_s = {ds:.1f}$',
        hovertemplate=
                'probability %{x:.2f}<br>'+
                'delta DC %{y:.1f}'+
                '<extra></extra>'
    ))


fig.add_trace(go.Scatter(
    x=2*[pf_norm], 
    y=[0,6],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=False, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-delta-dc-disadvantage-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-delta-dc-disadvantage-small')

In [6]:
# plots the effective increase in DC from automatically failing as a function of your chance to fail the save.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='fail save probability',
            range=[0.0, 1.0],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='$\Delta \, DC$',
            range=[0, 20],
            tick0=0, dtick=2.0,
            minor=dict(tick0=0, dtick=1.0),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.2,
            orientation='h',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
            itemclick=False, 
            itemdoubleclick=False,
            entrywidthmode='fraction',
            entrywidth=0.51,
        )
    )
)

# plot data
ds = 0.5
pf_norm = 0.65
pf = np.linspace(0.0, 1.00, 21)
for ds in [0.5, 0.0]:
    fig.add_trace(go.Scatter(
        x=pf, 
        y=20*(1 - save_damage(pf, ds)),
        mode='lines+markers', 
        marker=dict(size=8),
        unselected=dict(marker=dict(opacity=0.0, size=0)),
        selectedpoints=[13],
        name=f'$d_s = {ds:.1f}$',
        hovertemplate=
                'probability %{x:.2f}<br>'+
                'delta DC %{y:.2f}'+
                '<extra></extra>'
    ))


fig.add_trace(go.Scatter(
    x=2*[pf_norm], 
    y=[0,20],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-delta-dc-autofail-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-delta-dc-autofail-small')

In [9]:
# compares the stunned condition for PCs and monsters

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='monster CR (PC level)',
            range=[0.0, 20.0],
            tick0=0, dtick=5.0,
            minor=dict(tick0=0, dtick=1.0),
        ),
        yaxis=dict(
            title_text='damage per round',
            range=[0, 150],
            tick0=0, dtick=20,
            minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            yanchor='top',  y=1.00,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
            itemclick=False, itemdoubleclick=False,
            tracegroupgap=0,
        ),
    )
)

# plot data
CONDITIONS = {
    'Blinded':       {'Enemy Attacks': -0.37, 'Enemy Saves':  0.00, 'Ally Attacks':  0.37, 'Ally Saves':  0.00},
    'Charmed':       {'Enemy Attacks':  0.00, 'Enemy Saves':  0.00, 'Ally Attacks':  0.00, 'Ally Saves':  0.00},
    'Deafened':      {'Enemy Attacks':  0.00, 'Enemy Saves':  0.00, 'Ally Attacks':  0.00, 'Ally Saves':  0.00},
    'Frightened':    {'Enemy Attacks': -0.37, 'Enemy Saves':  0.00, 'Ally Attacks':  0.00, 'Ally Saves':  0.00},
    'Grappled':      {'Enemy Attacks':  0.00, 'Enemy Saves':  0.00, 'Ally Attacks':  0.00, 'Ally Saves':  0.00},
    'Incapacitated': {'Enemy Attacks': -1.00, 'Enemy Saves': -1.00, 'Ally Attacks':  0.37, 'Ally Saves':  0.00},
    'Invisible':     {'Enemy Attacks': -0.37, 'Enemy Saves':  0.00, 'Ally Attacks':  0.37, 'Ally Saves':  0.00},
    'Paralyzed':     {'Enemy Attacks': -1.00, 'Enemy Saves': -1.00, 'Ally Attacks':  0.98, 'Ally Saves':  0.21}, #  21-54%
    'Petrified':     {'Enemy Attacks': -1.00, 'Enemy Saves': -1.00, 'Ally Attacks': -0.35, 'Ally Saves': -0.27}, # -27-55%
    'Poisoned':      {'Enemy Attacks': -0.37, 'Enemy Saves':  0.00, 'Ally Attacks':  0.00, 'Ally Saves':  0.00},
    'Prone':         {'Enemy Attacks': -0.37, 'Enemy Saves':  0.00, 'Ally Attacks':  0.37, 'Ally Saves':  0.00},
    'Restrained':    {'Enemy Attacks': -0.37, 'Enemy Saves':  0.00, 'Ally Attacks':  0.37, 'Ally Saves':  0.14}, #  14-35%
    'Stunned':       {'Enemy Attacks': -1.00, 'Enemy Saves': -1.00, 'Ally Attacks':  0.37, 'Ally Saves':  0.21}, #  21-54%
    'Unconscious':   {'Enemy Attacks': -1.00, 'Enemy Saves': -1.00, 'Ally Attacks':  0.98, 'Ally Saves':  0.21}, #  21-54%
}

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD = dfD[dfD['CR'].between(1, 20)]
n_targets = 1
n_pcs = 4
n_mons = 1
pc_DPR = dfD['HP Mean']/(4*2.5)
mon_DPR = 0.65*dfD['DPR Mean']

colors = ['#7f7f7f','#1f77b4','#ff7f0e','#2ca02c']

fig.add_trace(go.Scatter(
    x=dfD['CR'], 
    y=pc_DPR,
    mode='lines', 
    line_color=colors[1],
    name='DPR',
    legendgroup='player character',
    hovertemplate=
            '<b>Player Characters</b><br>'+
            'level %{x:.0f}<br>'+
            'DPR %{y:.1f}'+
            '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=dfD['CR'], 
    y=mon_DPR,
    mode='lines', 
    line_color=colors[2],
    name='DPR',
    legendgroup='monster',
    hovertemplate=
            '<b>Monsters</b><br>'+
            'CR %{x:.0f}<br>'+
            'DPR %{y:.1f}'+
            '<extra></extra>'
))

for cond in ['Stunned']:
    c = CONDITIONS[cond]

    mon_weight = -c['Enemy Attacks']
    pc_weight = 0.5*(c['Ally Attacks'] + c['Ally Saves'])
    fig.add_trace(go.Scatter(
        x=dfD['CR'], 
        y=n_targets*mon_weight*mon_DPR + n_pcs*pc_weight*pc_DPR,
        mode='lines', 
        line_dash='dash',
        line_color=colors[1],
        name=f'{cond.lower()}',
        legendgroup='player character',
        legendgrouptitle_text="player character",
        hovertemplate=
                '<b>Player Characters</b><br>'+
                'level %{x:.0f}<br>'+
                'DPR + healing %{y:.1f}'+
                '<extra></extra>'
    ))
    
    pc_weight = -c['Enemy Attacks']
    mon_weight = 0.5*(c['Ally Attacks'] + c['Ally Saves'])
    fig.add_trace(go.Scatter(
        x=dfD['CR'], 
        y=n_mons*mon_weight*mon_DPR + n_targets*pc_weight*pc_DPR,
        mode='lines', 
        line_dash='dash',
        line_color=colors[2],
        name=f'{cond.lower()}',
        legendgroup='monster',
        legendgrouptitle_text="monster",
        hovertemplate=
                '<b>Monsters</b><br>'+
                'CR %{x:.0f}<br>'+
                'DPR + healing %{y:.1f}'+
                '<extra></extra>'
    ))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dpr-stunned-illustrative-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dpr-stunned-illustrative-small')

In [10]:
# compares single target spell damage budget against typical monster damage

def cantrip_damage(lvl, targets='single', save_for_half=False):
    """Returns cantrip damage dice equation.
    lvl -- spellcasting level between 1 and 20
    targets -- either 'single' or 'multiple'
    save_for_half -- True if the cantrip deals half damage on save.
    """
    n = np.sum(np.array([1,5,11,17]) <= lvl)
    if targets == 'single':
        if save_for_half:
            damage = '{:d}d10'.format(n)
        else:
            damage = '{:d}d6'.format(2*n)
    elif targets == 'multiple':
        if save_for_half:
            damage = '{:d}d6'.format(n)
        else:
            damage = '{:d}d8'.format(n)

    return damage

def spell_damage_list(lvl, targets='single', save_for_half=False):
    ctrpDmg = cantrip_damage(lvl, targets, save_for_half)
    if targets == 'single':
        if save_for_half:
            damage_list = [ctrpDmg,'2d10','3d10', '5d10', '6d10', '8d10','10d10','11d10','12d10','15d10']
        else:
            damage_list = [ctrpDmg, '4d6', '6d6', '10d6', '12d6', '16d6', '20d6', '22d6', '24d6', '30d6']
    elif targets == 'multiple':
        if save_for_half:
            damage_list = [ctrpDmg,'2d6','4d6','6d6','7d6', '8d6','11d6','12d6','13d6','14d6']
        else:
            damage_list = [ctrpDmg,'2d8','4d8','6d8','7d8', '8d8','11d8','12d8','13d8','14d8']
    
    return damage_list

def spell_damage_average(sp_lvl=0, lvl=1, targets='single', save_for_half=False):
    damage_list = spell_damage_list(lvl, targets, save_for_half)
    return deq.roll_average(damage_list[sp_lvl])


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='monster CR (PC level)',
            range=[0.0, 20.0],
            tick0=0, dtick=5.0,
            minor=dict(tick0=0, dtick=1.0),
        ),
        yaxis=dict(
            title_text='damage per round',
            range=[0, 150],
            tick0=0, dtick=20,
            minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
        )
    )
)

colors = ['#7f7f7f','#1f77b4','#ff7f0e','#2ca02c']
levels = [max(1, sp_lvl*2-1) for sp_lvl in range(1, 10)]
sp_levels = [sp_lvl for sp_lvl in range(1, 10)]
sp_damage = [spell_damage_average(sp_lvl, max(1, sp_lvl*2-1)) for sp_lvl in range(1, 10)]
fig.add_trace(go.Scatter(
    x=levels, 
    y=sp_damage,
    mode='markers', 
    line_color=colors[1],
    customdata=sp_levels,
    name=f'single target spells',
    hovertemplate=
            '<b>Single Target Spell</b><br>'+
            'PC level %{x:.0f}<br>'+
            'spell level %{customdata:.0f}<br>'+
            'DPR %{y:.1f}'+
            '<extra></extra>'
))

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
fig.add_trace(go.Scatter(
    x=dfD['CR'], 
    y=dfD['DPR Mean'],
    mode='lines', 
    line_color=colors[2],
    name=f'monsters',
    hovertemplate=
            '<b>Monsters</b><br>'+
            'CR %{x:.0f}<br>'+
            'DPR %{y:.1f}'+
            '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-spells-vs-monsters-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-spells-vs-monsters-small')